In [53]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go


Connect cassandra

In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- system_auth
- system_schema
- system_distributed
- my_keyspace
- system
- settrade_api
- system_traces
- data_quran


Connenct Setrade API

In [78]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
                app_id="JcpuaKNsswC7BQ2d",                                 
                app_secret="fAwSncOQmYTdMlko3oQEWZLAfAcZCBAPlSQ1LuNeqE8=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="mama-D")            

account_info = deri.get_account_info()

account_info                                              

SettradeError: Service unavailable, Please try again during 05:15 to 04:15

In [14]:
#GetQuoteSymbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 162500, 'totalSellVolume': 42400, 'totalNoSideVolume': 187900, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': 1.34277, 'pe': 38.73, 'pbv': 5.98, 'percentYield': 1.5199999809265137, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [26]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="BBL",
interval="1d",
normalized=True,)
res

{'lastSequence': 698263,
 'time': [1677171600,
  1677430800,
  1677517200,
  1677603600,
  1677690000,
  1677776400,
  1678122000,
  1678208400,
  1678294800,
  1678381200,
  1678640400,
  1678726800,
  1678813200,
  1678899600,
  1678986000,
  1679245200,
  1679331600,
  1679418000,
  1679504400,
  1679590800,
  1679850000,
  1679936400,
  1680022800,
  1680109200,
  1680195600,
  1680454800,
  1680541200,
  1680627600,
  1680800400,
  1681059600,
  1681146000,
  1681232400,
  1681664400,
  1681750800,
  1681837200,
  1681923600,
  1682010000,
  1682269200,
  1682355600,
  1682442000,
  1682528400,
  1682614800,
  1682960400,
  1683046800,
  1683478800,
  1683565200,
  1683651600,
  1683738000,
  1683824400,
  1684083600,
  1684170000,
  1684256400,
  1684342800,
  1684429200,
  1684688400,
  1684774800,
  1684861200,
  1684947600,
  1685034000,
  1685293200,
  1685379600,
  1685466000,
  1685552400,
  1685638800,
  1685984400,
  1686070800,
  1686157200,
  1686243600,
  1686502800,
 

In [77]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
limit=1,
normalized=True,)
res

SettradeError: The access token is invalid or has expired

In [75]:
# 👀 ตรวจสอบก่อนว่า res เป็น dict และมี key 'data' หรือไม่
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,698257,1677603600,73.00,73.50,72.50,72.50,13912341,1.013501e+09
1,698257,1677690000,72.50,72.50,71.25,71.75,34605326,2.487860e+09
2,698257,1677776400,71.50,71.50,70.25,70.50,25367513,1.791836e+09
3,698257,1678122000,70.75,71.00,69.75,69.75,26164171,1.837748e+09
4,698257,1678208400,69.25,70.00,69.00,69.75,32814418,2.278454e+09
...,...,...,...,...,...,...,...,...
495,698257,1742403600,65.25,70.00,65.00,70.00,4405101,2.908413e+08
496,698257,1742490000,69.00,91.25,59.25,67.00,4222400,2.981925e+08
497,698257,1742749200,64.75,65.25,64.00,65.25,2254101,1.456885e+08
498,698257,1742835600,52.00,52.00,50.00,52.00,314400,1.621875e+07


In [64]:
df.columns

Index(['lastSequence', 'time', 'open', 'high', 'low', 'close', 'volume',
       'value'],
      dtype='object')

In [74]:
# แปลงวันที่ให้เป็น datetime object
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# วาดกราฟ
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))

fig.update_layout(
    title="กราฟแท่งเทียนหุ้น (จาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)

fig.show()


KeyError: 'time'

In [73]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ให้แสดงกราฟใน browser
pio.renderers.default = 'browser'

# แปลงข้อมูล JSON เป็น DataFrame
df = pd.DataFrame(res)
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))

fig.update_layout(
    title="กราฟแท่งเทียนหุ้น (จาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)

fig.show()
